<a href="https://colab.research.google.com/github/SanjayHaritwal/AI_TASK_TTS_STT/blob/main/tesla_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras pandas pandas-datareader numpy matplotlib scikit-learn

In [3]:
import keras
from keras.layers import Dense, LSTM, Dropout
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np
from matplotlib import style
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

style.use('ggplot')

start = datetime.datetime(2014, 1, 1)
end = datetime.datetime(2018, 12, 30)
df = web.DataReader("TSLA", 'yahoo', start, end)

start = datetime.datetime(2019, 1, 1)
end = datetime.date.today()
test_df = web.DataReader("TSLA", 'yahoo', start, end)

df = df.sort_values('Date')
test_df = test_df.sort_values('Date')

# Fix the date index
df.reset_index(inplace=True)
df.set_index("Date", inplace=True)
test_df.reset_index(inplace=True)
test_df.set_index("Date", inplace=True)

plt.figure(figsize=(12, 6))
plt.plot(df["Adj Close"])
plt.xlabel('Date', fontsize=15)
plt.ylabel('Adjusted Close Price', fontsize=15)
plt.show()

close_px = df['Adj Close']
mavg = close_px.rolling(window=100).mean()

plt.figure(figsize=(12, 6))
close_px.plot(label='TSLA')
mavg.plot(label='mavg')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# Convert dates to integers for training
import matplotlib.dates as mdates

dates_df = df.copy()
dates_df = dates_df.reset_index()

org_dates = dates_df['Date']

# Convert dates to integers
dates_df['Date'] = dates_df['Date'].map(mdates.date2num)

dates = dates_df['Date'].values
prices = df['Adj Close'].values

dates = np.reshape(dates, (len(dates), 1))
prices = np.reshape(prices, (len(prices), 1))

svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(dates, prices)

plt.figure(figsize=(12, 6))
plt.plot(dates, prices, color='black', label='Data')
plt.plot(org_dates, svr_rbf.predict(dates), color='red', label='RBF model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

train_data = df.loc[:, 'Adj Close'].values
print(train_data.shape)

scaler = MinMaxScaler()
train_data = train_data.reshape(-1, 1)
scaler.fit(train_data)
train_data = scaler.transform(train_data)

def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Create training data for LSTM
time_steps = 36
X_train, y_train = create_dataset(train_data, time_steps)

# Reshape data for LSTM input
X_train = np.reshape(X_train, (X_train.shape[0], 36, 1))

# Build the LSTM model
model = keras.Sequential()

model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=100))
model.add(Dropout(0.2))

model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, epochs=20, batch_size=10, validation_split=0.30)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

test_data = test_df['Adj Close'].values
test_data = test_data.reshape(-1, 1)
test_data = scaler.transform(test_data)

X_test, y_test = create_dataset(test_data, time_steps)

y_test = y_test.reshape(-1, 1)
org_y = scaler.inverse_transform(y_test)

X_test = np.reshape(X_test, (X_test.shape[0], 36, 1))

predicted_y = model.predict(X_test)
predicted_y = scaler.inverse_transform(predicted_y)

plt.plot(org_y, color='red', label='Real Tesla Stock Price')
plt.plot(predicted_y, color='blue', label='Predicted Tesla Stock Price')
plt.title('Tesla Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Tesla Stock Price')
plt.legend()
plt.show()

AttributeError: 'NoneType' object has no attribute 'group'